In [258]:
from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec
import json, yaml

In [259]:
def get_api_operation_by_id(json_spec, s_operation_id):
    paths = json_spec.dict_.get("paths", {})
    for path, methods in paths.items():
        for method, operation in methods.items():
            # Check if 'operation' is a dictionary
            if isinstance(operation, dict):
                operation_id = operation.get("operationId")
                
                if operation_id == s_operation_id:
                    return operation, method
            else:
                # Handle the case where 'operation' is not a dictionary
                # print(f"Skipping invalid operation: {operation}")
                pass


In [263]:
import re

def resolve_refs(input_dict, json_spec):
    # Check if the input_dict is a dictionary and contains a '$ref' key
    if isinstance(input_dict, dict) and '$ref' in input_dict:
        ref_value = input_dict['$ref']
        
        # Use regular expression to extract the schema name
        match = re.match(r'#/components/schemas/(\w+)', ref_value)
        if match:
            schema_name = match.group(1)
            
            # Try to find the corresponding schema in the json_spec dictionary
            schema = json_spec.dict_.get("components", {}).get("schemas", {}).get(schema_name)
            
            # If a matching schema is found, replace the '$ref' key with the schema
            if schema:
                return schema
        
    # Recursively process nested dictionaries and lists
    if isinstance(input_dict, dict):
        for key, value in input_dict.items():
            input_dict[key] = resolve_refs(value, json_spec)
    elif isinstance(input_dict, list):
        for i, item in enumerate(input_dict):
            input_dict[i] = resolve_refs(item, json_spec)
    
    return input_dict

import re

# ref_list -> [{'$ref': '#/components/parameters/QueryAlbumIds'},
#  {'$ref': '#/components/parameters/QueryMarket'}]

def hydrateParams(json_spec, ref_list):
    last_portion_list = []
    
    for ref in ref_list:
        if '$ref' in ref:  # Check if '$ref' exists in the dictionary
            match = re.search(r'/([^/]+)$', ref['$ref'])
            if match:
                last_portion_list.append(json_spec["components"]["parameters"][match.group(1)])
        else:
            # If '$ref' doesn't exist, add the reference as is
            last_portion_list.append(ref)
    
    return last_portion_list

In [261]:
def process_api_operation(method, api_operation, json_spec):
    if method == "get":
        return api_operation
    
    request_body = api_operation.get("requestBody")
    
    if not isinstance(request_body, dict):
        return api_operation  # Return the request_schema if it's not a dictionary
    
    request_schema = request_body["content"]["application/json"]["schema"]
    resolved_request = resolve_refs(request_schema, json_spec.dict_)
    return resolved_request

In [264]:
with open("openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=data, max_value_length=4000)

api_operation, method=get_api_operation_by_id(json_spec, "pause-a-users-playback")
isolated_request = process_api_operation(method, api_operation, json_spec)

print(isolated_request["parameters"])
if isolated_request and isolated_request["parameters"]:
    isolated_request["parameters"] = hydrateParams(json_spec.dict_, isolated_request["parameters"])

isolated_request

[{'in': 'query', 'name': 'device_id', 'required': False, 'schema': {'description': "The id of the device this command is targeting. If not supplied, the user's currently active device is the target.\n", 'example': '0d1841b0976bae2a3a310dd74c0f3df354899bc8', 'title': 'Device ID', 'type': 'string'}}]


{'description': "Pause playback on the user's account.\n",
 'operationId': 'pause-a-users-playback',
 'parameters': [{'in': 'query',
   'name': 'device_id',
   'required': False,
   'schema': {'description': "The id of the device this command is targeting. If not supplied, the user's currently active device is the target.\n",
    'example': '0d1841b0976bae2a3a310dd74c0f3df354899bc8',
    'title': 'Device ID',
    'type': 'string'}}],
 'responses': {'204': {'description': 'Playback paused'},
  '401': {'$ref': '#/components/responses/Unauthorized'},
  '403': {'$ref': '#/components/responses/Forbidden'},
  '429': {'$ref': '#/components/responses/TooManyRequests'}},
 'security': [{'oauth_2_0': ['user-modify-playback-state']}],
 'summary': 'Pause Playback\n',
 'tags': ['Player'],
 'x-spotify-docs-console-url': '/console/put-pause/',
 'x-spotify-docs-endpoint-name': "Pause a User's Playback",
 'x-spotify-policy-list': {'$ref': '#/components/x-spotify-policy/playerPolicyList'}}